# サンプルコード中の対角行列化について

サンプルコード中では，行列 `X` の SVD から得られる `U, S, VT` の掛け算をしたいときに，
S を対角行列化しています．これは
`U @ S @ VT` における `@` が（通常の数学の文脈における）行列の掛け算になっているからです．

SVD の出力する `S` のサイズは $r$ ですが，これを対角行列化（以降，これを `matS` と呼ぶことにします）するとサイズは $r\times r$ になってしまいます．対角行列のためのメモリを確保する時間や，メモリの無駄遣いを減らす方法を考えてみましょう．

### numpy.einsum

numpy.einsumメソッドは，アインシュタインの縮約記法に基づいてベクトルや行列に関するさまざまな演算を実現します．
標準的なベクトル・行列間の演算のためのメソッドは numpy があらかじめ用意してくれていますが，そこから漏れた素朴な演算を実現するときに便利です．

次のコードが示すように，`np.einsum("...,...j",S,VT)` は `matS @VT` と等しい結果を与えます．したがって，これを利用することで `matS` を経由せずに `U @ S @ VT` を計算することができます．

In [43]:
import numpy as np
rs = np.random.RandomState(0)

X= rs.rand(100,100)
U, S, VT = np.linalg.svd(X,full_matrices=False)

matS = np.diag(S)

print("error = ", np.sum(np.einsum("...,...j",S,VT) - matS @ VT))

error =  0.0


計算にかかっている時間を計測してみましょう．次の二つのコードセルが示すように，`numpy.einsum` を用いることで処理が高速化されています．

In [89]:
%%timeit
matS = np.diag(S)
U @ matS @VT

49 µs ± 2.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [86]:
%%timeit
U @ np.einsum("...,...j",S,VT)

32.3 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


`matS` を導入する時間を計測時間から除いてみましょう．これでも `numpy.einsum` を用いた場合より遅いっぽいです．

In [88]:
%%timeit
#matS = np.diag(S)
U @ matS @ VT

40.4 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### reshape とブロードキャスト

`S` を列ベクトルに `reshape` してからブロードキャストしても同じことができます．  
`@` は `*` よりも演算の優先順位が高いので，括弧をつける必要があります．

In [65]:
print("error = ", np.sum(U @ (np.reshape(S,(-1,1)) * VT) - U @ matS @ VT))

error =  1.9647071513098635e-14


これが一番高速になるっぽいですね．`reshape` はできる限り配列のビューを返すので，メモリも浪費していないはずです．

In [75]:
%%timeit
U @ (np.reshape(S,(-1,1)) * VT)

28.4 µs ± 783 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
